In [1]:
import pandas as pd
from common_fun import *
from datetime import datetime, timedelta
import numpy as np

# Read parquet file

In [2]:
file_path = "../data/eng_1516_events.parquet"

In [3]:
df = pd.read_parquet(file_path)

### Saving column names

In [4]:
column_list = df.columns.tolist()

### Utility functions

In [5]:
players_df = df[["player_id", "player"]].drop_duplicates().dropna()

# Compute total time played by each player

## Get total time played each game

In [6]:
def timestamp_to_timedelta(timestamp_str: str) -> timedelta:
    """Convert timestamp string to timedelta object."""
    ts = datetime.strptime(timestamp_str, "%H:%M:%S.%f")
    return timedelta(hours=ts.hour, minutes=ts.minute, seconds=ts.second, microseconds=ts.microsecond)


def process_substitutions(subs_df: pd.DataFrame, players_data: dict, out_sub: set, in_sub: dict):
    """Process substitutions for a given period and update player data."""
    if subs_df.empty:
        return
    
    for i, row in subs_df.iterrows():
        curr_p_id = row.player_id
        curr_ts = timestamp_to_timedelta(row.timestamp)
        
        # Add time played for outgoing player
        players_data[curr_p_id]['time_played'] += curr_ts
        players_data[curr_p_id]['is_sub_out'] = True
        out_sub.add(curr_p_id)
        
        # Add incoming substitute to players_data
        sub_id = row.substitution_replacement_id
        players_data[sub_id] = {
            'position_id': 0,
            'time_played': timedelta(),
            'is_starting': False,
            'is_sub_in': True,
            'is_sub_out': False
        }
        in_sub[sub_id] = curr_ts


def update_remaining_players_time(players_data: dict, period_end_ts: timedelta, out_sub: set, in_sub: dict):
    """Update time played for players who weren't substituted out during the period."""
    for p_id, p_data in players_data.items():
        if p_id not in out_sub:
            if p_id in in_sub:
                p_data['time_played'] += period_end_ts - in_sub[p_id]
            else:
                p_data['time_played'] += period_end_ts


def get_match_player_minutes(match_data: pd.DataFrame):
    players_data = {}
    match_id = match_data.iloc[0]['match_id']
    teams_list = match_data['team_id'].unique().tolist()
    
    # Initialize starting lineup
    for team in teams_list:
        curr_team_lineup = match_data[
            (match_data['type'] == 'Starting XI') &
            (match_data['team_id'] == team)
        ].tactics.values[0]['lineup']
        for player in curr_team_lineup:
            p_id = player['player']['id']
            p_jn = player['jersey_number']
            p_pos_id = player['position']['id']
            players_data[p_id] = {
                'jersey_number': p_jn,
                'position_id': p_pos_id,
                'time_played': timedelta(),
                'is_starting': True,
                'is_sub_in': False,
                'is_sub_out': False
            }
    
    subs = match_data[match_data['type'] == 'Substitution'].dropna(axis=1)
    half_ends = match_data[match_data['type'] == 'Half End'].dropna(axis=1)
    out_sub = set()
    in_sub = {}
    
    # Process first half substitutions
    fh_subs = subs[subs['period'] == 1]
    process_substitutions(fh_subs, players_data, out_sub, in_sub)
    
    # Update remaining players' time at first half end
    fh_end_ts = timestamp_to_timedelta(half_ends[half_ends['period'] == 1].iloc[0].timestamp)
    update_remaining_players_time(players_data, fh_end_ts, out_sub, in_sub)
    
    # Process second half substitutions
    sh_subs = subs[subs['period'] == 2]
    process_substitutions(sh_subs, players_data, out_sub, in_sub)
    
    # Update remaining players' time at second half end
    sh_end_ts = timestamp_to_timedelta(half_ends[half_ends['period'] == 2].iloc[0].timestamp)
    update_remaining_players_time(players_data, sh_end_ts, out_sub, in_sub)
    
    # Create final DataFrame
    match_time_played = pd.DataFrame(
        data=players_data.values(),
        index=list(players_data.keys())
    )
    match_time_played['match_id'] = match_id
    match_time_played = match_time_played.reset_index(names='player_id')
    match_time_played['player_id'] = match_time_played['player_id'].astype('int')
    
    return match_time_played

## Get total playing time over all matches for each player

In [7]:
def get_player_minutes(event_data: pd.DataFrame) -> pd.DataFrame:
    player_minutes = pd.DataFrame()
    for match_id, match_data in event_data.groupby(by='match_id'):
        curr_md = get_match_player_minutes(match_data=match_data)
        player_minutes = pd.concat([player_minutes, curr_md])
    return player_minutes

In [8]:
players_minutes = get_player_minutes(event_data=df)

### Save playing time over season

*time_played* in minutes

In [9]:
tot_player_time_path = '../data/tot_player_time.csv'

In [10]:
tot_player_time = players_minutes.groupby(by='player_id') \
                                    .agg({'time_played':'sum'}).reset_index() \
                                    .sort_values(by='time_played', ascending=False)
tot_player_time['time_played'] = tot_player_time['time_played'].apply(lambda tp: tp.total_seconds() / 60)

In [11]:
tot_player_time.to_csv(tot_player_time_path, index=False)

### Save playing time per match

*time_played* in seconds

In [12]:
playing_time_match_path = '../data/players_playing_time_match.csv'

In [13]:
players_minutes['time_played'] = players_minutes['time_played'].apply(lambda tp: tp.total_seconds())
players_minutes.to_csv(playing_time_match_path, index=False)